# Logistic Regression Consulting Project

## Binary Customer Churn

A marketing agency has many customers that use their service to produce ads for the client/customer websites. They've noticed that they have quite a bit of churn in clients. They basically randomly assign account managers right now, but want you to create a machine learning model that will help predict which customers will churn (stop buying their service) so that they can correctly assign the customers most at risk to churn an account manager. Luckily they have some historical data, can you help them out? Create a classification algorithm that will help classify whether or not a customer churned. Then the company can test this against incoming data for future customers to predict which customers will churn and assign them an account manager.

The data is saved as customer_churn.csv. Here are the fields and their definitions:

    Name : Name of the latest contact at Company
    Age: Customer Age
    Total_Purchase: Total Ads Purchased
    Account_Manager: Binary 0=No manager, 1= Account manager assigned
    Years: Totaly Years as a customer
    Num_sites: Number of websites that use the service.
    Onboard_date: Date that the name of the latest contact was onboarded
    Location: Client HQ Address
    Company: Name of Client Company
    
Once you've created the model and evaluated it, test out the model on some new data (you can think of this almost like a hold-out set) that your client has provided, saved under new_customers.csv. The client wants to know which customers are most likely to churn given this data (they don't have the label yet).

In [1]:
import findspark

In [2]:
findspark.init('/home/richard/spark-3.2.0-bin-hadoop3.2')

In [3]:
from pyspark.sql import SparkSession

In [37]:
spark = SparkSession.builder.appName('project').getOrCreate()

In [5]:
data = spark.read.csv('customer_churn.csv',inferSchema=True,header=True)

In [6]:
data.columns

['Names',
 'Age',
 'Total_Purchase',
 'Account_Manager',
 'Years',
 'Num_Sites',
 'Onboard_date',
 'Location',
 'Company',
 'Churn']

In [7]:
data.printSchema()

root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: string (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)
 |-- Churn: integer (nullable = true)



In [8]:
from pyspark.sql.types import TimestampType
data = data.withColumn('Onboard_date', data['Onboard_date'].cast(TimestampType()))

In [9]:
from pyspark.sql.functions import year

In [10]:
data=data.withColumn('Year',year(data['Onboard_date']))

In [11]:
data=data.select('Age',
 'Total_Purchase',
 'Years',
 'Num_Sites',
 'Churn',
 'Year')

In [12]:
data=data.na.drop()

In [13]:
from pyspark.ml.feature import VectorAssembler

In [14]:
assembler = VectorAssembler(inputCols=['Age',
 'Total_Purchase',
 'Years',
 'Num_Sites',
 'Year'],outputCol='features')

In [15]:
final_data = assembler.transform(data)
final_data = final_data.select('Churn','features')

In [16]:
from pyspark.ml.classification import LogisticRegression

In [17]:
model = LogisticRegression(labelCol='Churn')

In [18]:
train_data,test_data = final_data.randomSplit([0.7,0.3])

In [38]:
fitted_model = model.fit(train_data)

In [20]:
pred_model = fitted_model.evaluate(test_data)

In [21]:
pred_model.accuracy

0.889273356401384

In [22]:
pred_model.precisionByLabel

[0.928, 0.6410256410256411]

In [23]:
pred_model.areaUnderROC

0.8987521270561522

In [24]:
pred_model.recallByLabel

[0.943089430894309, 0.5813953488372093]

In [26]:
holdout_set = spark.read.csv('new_customers.csv',header=True,inferSchema=True)

In [28]:
holdout_set = holdout_set.select('Age',
 'Total_Purchase','Years',
 'Num_Sites','Onboard_date')

In [29]:
holdout_set = holdout_set.withColumn('Onboard_date', holdout_set['Onboard_date'].cast(TimestampType()))

In [30]:
holdout_set=holdout_set.withColumn('Year',year(holdout_set['Onboard_date']))

In [31]:
holdout_set = holdout_set.select('Age',
 'Total_Purchase','Years',
 'Num_Sites','Year')

In [32]:
holdout_set = holdout_set.na.drop()

In [33]:
assembler = VectorAssembler(inputCols=['Age',
 'Total_Purchase',
 'Years',
 'Num_Sites',
 'Year'],outputCol='features')

In [34]:
holdout_set = assembler.transform(holdout_set)

In [35]:
holdout_set = holdout_set.select('features')

In [36]:
fitted_model.transform(holdout_set).show()

+--------------------+--------------------+--------------------+----------+
|            features|       rawPrediction|         probability|prediction|
+--------------------+--------------------+--------------------+----------+
|[37.0,9935.53,7.7...|[2.13886154714560...|[0.89462333397360...|       0.0|
|[23.0,7526.94,9.2...|[-5.9998313594254...|[0.00247303914508...|       1.0|
|[65.0,100.0,1.0,1...|[-2.5201194272233...|[0.07445971436507...|       1.0|
|[32.0,6487.5,9.4,...|[-5.3465815432412...|[0.00474181815658...|       1.0|
|[32.0,13147.71,10...|[0.64369383605290...|[0.65558797930538...|       0.0|
|[22.0,8445.26,3.4...|[-1.0835063101883...|[0.25284305367890...|       1.0|
+--------------------+--------------------+--------------------+----------+

